In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

In [96]:
training_labels = pd.read_csv("labels_training_set.csv", low_memory=False)
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [101]:
training_labels = training_labels[['person','label']]
training_labels['label'].value_counts()

0    18434
1      980
Name: label, dtype: int64

In [105]:
data = data[['person','days_since_last_session', 'sessions']]
data.head()

,person,days_since_last_session,sessions
0,0008ed71,187,3
1,00091926,173,34
2,00091a7a,239,1
3,000ba417,178,5
4,000c79fe,175,1


In [111]:
features = pd.merge(training_labels, data, on='person', how='inner')
features = features.fillna(0)
features['label'].value_counts()
features = pd.concat([(features[features['label'] == 0].sort_values(by='days_since_last_session', ascending=True).head(5000)),(features[features['label'] == 1])])
features['label'].value_counts()

0    5000
1     980
Name: label, dtype: int64

In [112]:
train, test = train_test_split(features,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(features.columns)
features.remove('person')
features.remove('label')

x_train=train[features]
y_train=train['label']

x_test=test[features]
y_test=test['label']

Train:  5382 Test:  598


In [113]:
set_test = pd.merge(data,labels_predict,on = "person", how = "right")
personas = set_test["person"]
set_test = set_test.drop(columns=['person'])

In [114]:
def KNN(k):
    knn = KNeighborsClassifier(n_neighbors=k,metric='euclidean')
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    print (accuracy_score(pred, y_test))
    preds_kaggel_knn = knn.predict(set_test)
    preds_kaggel_knn = preds_kaggel_knn.tolist()
    df_kaggel_knn = pd.DataFrame(preds_kaggel_knn)
    df_kaggel_knn["person"] = personas
    df_kaggel_knn.rename(columns = {0: 'label'},inplace = True) 
    df_kaggel_knn = df_kaggel_knn[['person','label']]
    print (df_kaggel_knn["label"].value_counts())

In [115]:
for x in range(1,15):
    print (x)
    KNN(x)

1
0.9063545150501672
1    13492
0     5923
Name: label, dtype: int64
2
0.9230769230769231
1    12518
0     6897
Name: label, dtype: int64
3
0.919732441471572
1    12616
0     6799
Name: label, dtype: int64
4
0.9230769230769231
1    12488
0     6927
Name: label, dtype: int64
5
0.9247491638795987
1    12520
0     6895
Name: label, dtype: int64
6
0.9247491638795987
1    12473
0     6942
Name: label, dtype: int64
7
0.9247491638795987
1    12528
0     6887
Name: label, dtype: int64
8
0.9214046822742475
1    12455
0     6960
Name: label, dtype: int64
9
0.9230769230769231
1    12523
0     6892
Name: label, dtype: int64
10
0.9214046822742475
1    12461
0     6954
Name: label, dtype: int64
11
0.9214046822742475
1    12481
0     6934
Name: label, dtype: int64
12
0.9180602006688964
1    12312
0     7103
Name: label, dtype: int64
13
0.9180602006688964
1    12327
0     7088
Name: label, dtype: int64
14
0.9163879598662207
1    12304
0     7111
Name: label, dtype: int64


In [116]:
knn = KNeighborsClassifier(n_neighbors=5,metric='euclidean')
knn.fit(x_train, y_train)
pred = knn.predict(x_test)
print (accuracy_score(pred, y_test))

0.9247491638795987


In [117]:
preds_kaggel_knn = knn.predict(set_test)

In [118]:
preds_kaggel_knnpreds_kag  = preds_kaggel_knn.tolist()
df_kaggel_knn = pd.DataFrame(preds_kaggel_knn)
df_kaggel_knn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19415 entries, 0 to 19414
Data columns (total 1 columns):
0    19415 non-null int64
dtypes: int64(1)
memory usage: 151.8 KB


In [119]:
# df_kaggel_knn = df_kaggel_knn.to_frame()# df_kagg 
df_kaggel_knn["person"] = personas
df_kaggel_knn.rename(columns = {0: 'label'},inplace = True) 
df_kaggel_knn = df_kaggel_knn[['person','label']]

In [120]:
df_kaggel_knn["label"].value_counts()


1    12520
0     6895
Name: label, dtype: int64

In [121]:
df_kaggel_knn.to_csv('ResultadosKNN.csv',index=False)